# Projeto 2 - Ciência dos Dados

Nome: _Marco Tulio Masselli Rainho Teixeira_

Nome: _Talissa Gonçalves Albertini_

## Classificador automático de qualidade de investimentos em ações


### Descrição da Base de Dados 

A base de dados ultilizada neste projeto é composta por indicadores financeiros coletados ao longo de vários anos da empresa Microsoft

* DATE: Dada de coleta das informações
* NET_INCOME: A diferença entre o lucro bruto e as despesas
* CF_FREE_CASH_FLOW: 
* CF_CASH_FROM_OPER:
* CF_CASH_FROM_INV_ACT:
* CF_CASH_FROM_FNC_ACT:
* CUR_RATIO:
* TOT_DEBT_TO_COM_EQY:
* RETURN_COM_EQY:

Indicadores do DataSet Diário
* DATE: Dada de coleta das informações
* CUR_MKT_CAP : Valor de mercado, valor de uma ação vezes o número total ed ações
* TURNOVER : Indica o quão rápido a empresa consegue arrecadar dinheito com o seu inventário
* PX LAST : Preço da ação 
* EQY_SH_OUT :
* PE RATIO :
* PX_TO_BOOK_RATIO :
* EQY_DPS :


### Técnica utilizada

### Objetivo do Projeto

Este projeto tem o objetivo realizar a análise de indicadores financeiros de uma determinada ação e responder a seguinte questão: Esta ação deve ser tem uma? Os critérios relevantes para responder essa questão é basicamente  a probabilidade dessa ação valorizar durante o próximo trimestre, dado que parte significante dos indicadores são coletados de 3 em 3 meses.

### 1. Preparação do ambiente do Jupyter


In [1]:
%matplotlib inline
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb


### 2. Carregando a base de dados com os indicadores

In [2]:
xls = pd.ExcelFile('Dados_BBG.xlsx')
df1 = pd.read_excel(xls, 'MSFT-Quarterly')
df2 = pd.read_excel(xls, 'MSFT-Daily')

In [3]:
dft = df1.iloc[6:133, 1:] 
dft.iloc[:, 0] = dft.iloc[:, 0].dt.date # rectify the date column

dfd = df2.iloc[:, :] 

### 3. Tratamento do banco de dados de treinamento

In [5]:
# Set the column's names 

indicators_names = df1.iloc[4,[1,3,5,7,9,11,13,15,17]].array  
dft.rename( columns={'Unnamed: 1':indicators_names[0]}, inplace=True )
dft.rename( columns={'Unnamed: 3':indicators_names[1]}, inplace=True )
dft.rename( columns={'Unnamed: 5':indicators_names[2]}, inplace=True )
dft.rename( columns={'Unnamed: 7':indicators_names[3]}, inplace=True )
dft.rename( columns={'Unnamed: 9':indicators_names[4]}, inplace=True )
dft.rename( columns={'Unnamed: 11':indicators_names[5]}, inplace=True )
dft.rename( columns={'Unnamed: 13':indicators_names[6]}, inplace=True )
dft.rename( columns={'Unnamed: 15':indicators_names[7]}, inplace=True )
dft.rename( columns={'Unnamed: 17':indicators_names[8]}, inplace=True )


In [6]:
indicators_names

<PandasArray>
[                'DATE',           'NET_INCOME',    'CF_FREE_CASH_FLOW',
    'CF_CASH_FROM_OPER', 'CF_CASH_FROM_INV_ACT', 'CF_CASH_FROM_FNC_ACT',
            'CUR_RATIO',  'TOT_DEBT_TO_COM_EQY',       'RETURN_COM_EQY']
Length: 9, dtype: object

In [7]:
# Get rid of unnamed columns thar remained
dft = dft.loc[:, ~dft.columns.str.contains('^Unnamed')]

In [8]:
# Rectify the data type
floats = ['NET_INCOME', 'CF_FREE_CASH_FLOW', 'CF_CASH_FROM_OPER', 'CF_CASH_FROM_INV_ACT', 'CF_CASH_FROM_FNC_ACT', 'CUR_RATIO',  'TOT_DEBT_TO_COM_EQY', 'RETURN_COM_EQY']

for indicator in floats:
    dft[indicator] = dft[indicator].astype(float)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [9]:
dft

,DATE,NET_INCOME,CF_FREE_CASH_FLOW,CF_CASH_FROM_OPER,CF_CASH_FROM_INV_ACT,CF_CASH_FROM_FNC_ACT,CUR_RATIO,TOT_DEBT_TO_COM_EQY,RETURN_COM_EQY
6,1988-09-30,36.582,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1988-12-31,47.485,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1989-03-31,41.119,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1989-06-30,45.352,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,1989-09-30,49.588,NaN,59.465,NaN,NaN,2.8538,8.2158,NaN
11,1989-12-31,74.476,NaN,69.540,NaN,NaN,3.2248,1.2991,NaN
12,1990-03-31,75.169,NaN,92.490,NaN,NaN,3.5930,6.1148,NaN
13,1990-06-30,79.953,NaN,56.779,NaN,NaN,3.8541,0.7076,NaN
14,1990-09-30,87.600,NaN,120.600,NaN,NaN,3.6171,1.0097,39.2820
15,1990-12-31,112.900,NaN,97.100,NaN,NaN,3.5446,1.5735,38.4404


In [10]:
price = dft['NET_INCOME']
mv = dft['CUR_RATIO']
cov_price_mv = price.corr(mv)
cov_price_mv

-0.4163161274531859